#### The Purpose of this Notebook is to display the Safety Ranges of H2S
It will be broken and partitioned into 3 categories
1. Per Monitor
2. Per Area
3. Overall

**Import Functions and Data**

In [0]:
from pyspark.sql import functions as F

silverConditions = spark.read.table("gasmonitordata.silver.monitorh2sandconditions")
silverInfo = spark.read.table("gasmonitordata.silver.monitorinformation")

**Calculate Data**

In [0]:
safetyRange = silverConditions.alias('c').join(silverInfo.alias('i'), silverConditions.monitorNum == silverInfo.monitorNum). \
    groupBy('c.monitorNum', 'i.groupNum'). \
    agg(
        F.round(F.sum(F.when(F.col("c.h2sReading") <= 3.9, 1).otherwise(0)) / F.count("*") * 100, 2).alias("percentSafe"),
        F.round(F.sum(F.when((F.col("c.h2sReading") >= 4.0) & (F.col("c.h2sReading") <= 9.9), 1).otherwise(0)) / F.count("*") * 100, 2).alias("percentHazardous"),
        F.round(F.sum(F.when(F.col("c.h2sReading") >= 10.0, 1).otherwise(0)) / F.count("*") * 100, 2).alias("percentDangerous")). \
    orderBy('c.monitorNum', 'i.groupNum'). \
    select('c.monitorNum', 'i.groupNum', 'percentSafe', 'percentHazardous', 'percentDangerous')

**Export Data**

In [0]:
safetyRange.write.mode('overwrite').saveAsTable("gasmonitordata.gold.safetyrange")